In [1]:
'''
S&P 500 is always changing
This piece of code aims to find out the stock ticker symbol that the existing companies since the date added
ref: https://pythonprogramming.net/sp500-company-price-data-python-programming-for-finance/?completed=/sp500-company-list-python-programming-for-finance/
ref: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
'''
import pandas as pd
import numpy as np

import bs4 as bs
import pickle
import requests
import time
import lxml
from datetime import datetime

def get_sp500_tickers(date_added):
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        if row.findAll('td')[6].text == '':
            ticker = [row.findAll('td')[0].text.strip(), row.findAll('td')[1].text, row.findAll('td')[6].text]
        else:
            if datetime.strptime(row.findAll('td')[6].text[0:10], '%Y-%m-%d') < date_added:
                ticker = [row.findAll('td')[0].text.strip(), row.findAll('td')[1].text, row.findAll('td')[6].text]
            else: 
                continue
        tickers.append(ticker)
    return tickers

In [3]:
date_added = datetime(2009, 11, 30)
stock_list = get_sp500_tickers(date_added)
print(len(stock_list))
sorted(stock_list)
# AAPL, GOOG, AMZN, MSFT, JPM, WMT, NKE, DIS

318


[['A', 'Agilent Technologies Inc', '2000-06-05'],
 ['AAPL', 'Apple Inc.', '1982-11-30'],
 ['ABC', 'AmerisourceBergen Corp', '2001-08-30'],
 ['ABT', 'Abbott Laboratories', '1964-03-31'],
 ['ADBE', 'Adobe Inc.', '1997-05-05'],
 ['ADI', 'Analog Devices, Inc.', '1999-10-12'],
 ['ADM', 'Archer-Daniels-Midland Co', '1981-07-29'],
 ['ADP', 'Automatic Data Processing', '1981-03-31'],
 ['ADSK', 'Autodesk Inc.', '1989-12-01'],
 ['AEE', 'Ameren Corp', '1991-09-19'],
 ['AEP', 'American Electric Power', ''],
 ['AES', 'AES Corp', '1998-10-02'],
 ['AFL', 'AFLAC Inc', '1999-05-28'],
 ['AGN', 'Allergan, plc', '1999-04-12'],
 ['AIG', 'American International Group', '1980-03-31'],
 ['AIV', 'Apartment Investment & Management', '2003-03-14'],
 ['AIZ', 'Assurant', '2007-04-10'],
 ['AKAM', 'Akamai Technologies Inc', '2007-07-12'],
 ['ALL', 'Allstate Corp', '1995-07-13'],
 ['AMAT', 'Applied Materials Inc.', '1995-03-16'],
 ['AMGN', 'Amgen Inc.', '1992-01-02'],
 ['AMP', 'Ameriprise Financial', '2005-10-03'],
 

In [13]:
# using stockstats library to get indicators
import yfinance as yf
data = yf.download("AAPL", start="2009-12-30", end="2019-12-31")

[*********************100%***********************]  1 of 1 completed


In [14]:
data['high_low_diff_ratio'] = (data['High'] - data['Low']) / data['Close']
data['open_close_diff_ratio'] = (data['Open'] - data['Close']) / data['Close']
data

,Open,High,Low,Close,Adj Close,Volume,high_low_diff_ratio,open_close_diff_ratio
Date,,,,,,,,
2009-12-29,30.375713,30.388571,29.818571,29.871429,25.929619,111301400,0.019082,0.016882
2009-12-30,29.832857,30.285715,29.758572,30.234285,26.244589,103021100,0.017435,-0.013277
2009-12-31,30.447144,30.478571,30.080000,30.104286,26.131752,88102700,0.013240,0.011389
2010-01-04,30.490000,30.642857,30.340000,30.572857,26.538483,123432400,0.009906,-0.002710
2010-01-05,30.657143,30.798571,30.464285,30.625713,26.584366,150476200,0.010915,0.001026
...,...,...,...,...,...,...,...,...
2019-12-23,280.529999,284.250000,280.369995,284.000000,283.327576,24643000,0.013662,-0.012218
2019-12-24,284.690002,284.890015,282.920013,284.269989,283.596924,12119700,0.006930,0.001478
2019-12-26,284.820007,289.980011,284.700012,289.910004,289.223602,23280300,0.018213,-0.017557


In [15]:
'''
ref: https://blog.csdn.net/freewebsys/article/details/78578548
'''
import stockstats
stock = stockstats.StockDataFrame.retype(data)

In [16]:
# BOLL needs diff with itself and close
# open - close
# high - low
# close - close_5_sma & close - close_10_sma 考慮不用
# defalut: atr:14, cci: 14
# dma: 0 - 50 moving average
stock[['close_1_d', 'close_7_d', 'close_30_d','change', 'open_delta','close_delta','volume_delta', 'close_-1_r','close_-6_r', 'boll', 'boll_ub', 'boll_lb', 'boll_-1_d', 'boll_ub_-1_d', 'boll_lb_-1_d' ,'kdjk','kdjd','kdjj', 'macd','macds','macdh', 'rsi_6', 'rsi_12', 'wr_6', 'wr_12', 'cci', 'atr', 'dma', 'vr']]
stock[['rsi_14', 'rsi_30']]

/Users/chinkashiwakin/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values


,rsi_14,rsi_30
Date,,
2009-12-29,NaN,NaN
2009-12-30,100.000000,100.000000
2009-12-31,72.159152,72.959648
2010-01-04,86.619425,86.535396
2010-01-05,87.413578,87.280582
...,...,...
2019-12-23,74.208465,70.183414
2019-12-24,74.436585,70.299662
2019-12-26,78.678994,72.607560


In [17]:
print(stock.columns.values)
print(len(stock.columns.values))

['open' 'high' 'low' 'close' 'adj close' 'volume' 'high_low_diff_ratio'
 'open_close_diff_ratio' 'close_1_s' 'close_1_d' 'close_7_s' 'close_7_d'
 'close_30_s' 'close_30_d' 'change' 'open_delta' 'close_delta'
 'volume_delta' 'close_-1_r' 'close_-6_r' 'close_20_sma' 'close_20_mstd'
 'boll' 'boll_ub' 'boll_lb' 'boll_-1_s' 'boll_-1_d' 'boll_ub_-1_s'
 'boll_ub_-1_d' 'boll_lb_-1_s' 'boll_lb_-1_d' 'rsv_9' 'kdjk_9' 'kdjk'
 'kdjd_9' 'kdjd' 'kdjj_9' 'kdjj' 'macd' 'macds' 'macdh' 'close_-1_s'
 'close_-1_d' 'rs_6' 'rsi_6' 'rs_12' 'rsi_12' 'wr_6' 'wr_12' 'middle'
 'middle_14_sma' 'cci' 'tr' 'atr' 'close_10_sma' 'close_50_sma' 'dma' 'vr'
 'rs_14' 'rsi_14' 'rs_30' 'rsi_30']
62


In [18]:
stock_slice = pd.DataFrame(stock)
stock_slice = stock_slice.drop(stock.index[:23])
stock_slice = stock_slice.drop(stock.index[-1])
stock_slice['boll_k_diff'] = stock_slice['boll'] - stock_slice['close']
stock_slice = stock_slice[['close_1_d', 'close_7_d', 'close_30_d','change', 'open_delta','close_delta','volume_delta', 'high_low_diff_ratio', 
'open_close_diff_ratio','close_-1_r','close_-6_r','kdjk','kdjd','kdjj', 'macd','macds', 
'macdh', 'rsi_6', 'rsi_12', 'rsi_30', 'rsi_14', 'wr_6', 'wr_12', 'cci', 'atr', 'dma', 'vr', 'boll_-1_d','boll_ub_-1_d', 'boll_lb_-1_d', 
'boll_k_diff']]
# stock_slice = stock_slice[['close_1_d', 'close_7_d', 'close_30_d','change', 'open_delta','close_delta','volume_delta', 'high_low_diff_ratio','kdjk','kdjd','kdjj', 'macd','macds', 
# 'macdh', 'rsi_12', 'wr_6', 'cci', 'atr', 'dma', 'vr', 'boll_-1_d','boll_ub_-1_d', 'boll_lb_-1_d', 
# 'boll_k_diff' ]]

In [19]:
# 1 means rise, 0 means fall
stock_slice['Y_1'] = stock_slice['close_1_d'].apply(lambda x: 1 if x <= 0 else 0)
stock_slice['Y_7'] = stock_slice['close_7_d'].apply(lambda x: 1 if x <= 0 else 0)
stock_slice['Y_30'] = stock_slice['close_30_d'].apply(lambda x: 1 if x <= 0 else 0)

In [20]:
label_1 = stock_slice['Y_1']
label_7 = stock_slice['Y_7']
label_30 = stock_slice['Y_30']
stock_slice = stock_slice.drop(['close_1_d', 'close_7_d', 'close_30_d','Y_1', 'Y_7', 'Y_30'], axis=1)
stock_slice
print(stock_slice.columns.values)
print(len(stock_slice.columns.values))

['change' 'open_delta' 'close_delta' 'volume_delta' 'high_low_diff_ratio'
 'open_close_diff_ratio' 'close_-1_r' 'close_-6_r' 'kdjk' 'kdjd' 'kdjj'
 'macd' 'macds' 'macdh' 'rsi_6' 'rsi_12' 'rsi_30' 'rsi_14' 'wr_6' 'wr_12'
 'cci' 'atr' 'dma' 'vr' 'boll_-1_d' 'boll_ub_-1_d' 'boll_lb_-1_d'
 'boll_k_diff']
28


In [21]:
stock_slice.describe()

,change,open_delta,close_delta,volume_delta,high_low_diff_ratio,open_close_diff_ratio,close_-1_r,close_-6_r,kdjk,kdjd,...,wr_6,wr_12,cci,atr,dma,vr,boll_-1_d,boll_ub_-1_d,boll_lb_-1_d,boll_k_diff
count,2494.000000,2494.000000,2494.000000,2.494000e+03,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,...,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000
mean,0.107103,0.105709,0.105045,-6.050626e+04,0.018510,0.000037,0.107103,0.632239,58.591791,58.543468,...,42.551094,40.369523,24.719764,2.211068,1.904366,120.079532,0.098186,0.104711,0.091661,-0.953906
std,1.617221,1.979966,1.938058,3.231543e+07,0.011117,0.012677,1.617221,3.960256,24.585077,21.762430,...,31.109013,31.042716,108.198633,1.242197,6.962697,61.245491,0.450313,0.708245,0.749434,4.903264
min,-12.355796,-15.559998,-15.729996,-1.863463e+08,0.004150,-0.080004,-12.355796,-13.082650,6.245540,9.751213,...,0.000000,0.000000,-339.128982,0.500494,-29.463201,26.554446,-2.375500,-5.432814,-4.373042,-17.176501
25%,-0.686855,-0.699997,-0.647141,-1.066952e+07,0.011460,-0.007445,-0.686855,-1.793125,37.194766,40.923357,...,13.053086,11.762234,-60.387432,1.373965,-1.023620,76.042434,-0.106768,-0.156072,-0.146764,-3.462124
50%,0.089511,0.120003,0.076427,-6.511500e+05,0.016143,-0.000456,0.089511,0.822144,62.408975,61.315206,...,37.931982,33.258232,42.635253,1.856660,1.703972,102.278385,0.091500,0.036374,0.041363,-0.996284
75%,0.978609,0.969997,0.916782,1.065308e+07,0.022676,0.007103,0.978609,3.054426,80.649761,77.089931,...,70.250174,68.177318,107.954761,2.692144,5.249286,149.269961,0.314464,0.375690,0.347381,1.265107
max,8.874137,16.239998,11.210007,3.930395e+08,0.239594,0.071043,8.874137,14.070968,97.575313,95.729289,...,100.000000,100.000000,368.904712,7.390591,25.016003,517.276520,1.636501,3.804334,4.703747,27.146007


In [22]:
label_1.describe()

count    2494.000000
mean        0.530072
std         0.499195
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: Y_1, dtype: float64

In [23]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, IsolationForest,VotingClassifier
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [27]:
stock_slice.describe()

,change,open_delta,close_delta,volume_delta,high_low_diff_ratio,open_close_diff_ratio,close_-1_r,close_-6_r,kdjk,kdjd,...,wr_6,wr_12,cci,atr,dma,vr,boll_-1_d,boll_ub_-1_d,boll_lb_-1_d,boll_k_diff
count,2494.000000,2494.000000,2494.000000,2.494000e+03,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,...,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000,2494.000000
mean,0.107103,0.105709,0.105045,-6.050626e+04,0.018510,0.000037,0.107103,0.632239,58.591791,58.543468,...,42.551094,40.369523,24.719764,2.211068,1.904366,120.079532,0.098186,0.104711,0.091661,-0.953906
std,1.617221,1.979966,1.938058,3.231543e+07,0.011117,0.012677,1.617221,3.960256,24.585077,21.762430,...,31.109013,31.042716,108.198633,1.242197,6.962697,61.245491,0.450313,0.708245,0.749434,4.903264
min,-12.355796,-15.559998,-15.729996,-1.863463e+08,0.004150,-0.080004,-12.355796,-13.082650,6.245540,9.751213,...,0.000000,0.000000,-339.128982,0.500494,-29.463201,26.554446,-2.375500,-5.432814,-4.373042,-17.176501
25%,-0.686855,-0.699997,-0.647141,-1.066952e+07,0.011460,-0.007445,-0.686855,-1.793125,37.194766,40.923357,...,13.053086,11.762234,-60.387432,1.373965,-1.023620,76.042434,-0.106768,-0.156072,-0.146764,-3.462124
50%,0.089511,0.120003,0.076427,-6.511500e+05,0.016143,-0.000456,0.089511,0.822144,62.408975,61.315206,...,37.931982,33.258232,42.635253,1.856660,1.703972,102.278385,0.091500,0.036374,0.041363,-0.996284
75%,0.978609,0.969997,0.916782,1.065308e+07,0.022676,0.007103,0.978609,3.054426,80.649761,77.089931,...,70.250174,68.177318,107.954761,2.692144,5.249286,149.269961,0.314464,0.375690,0.347381,1.265107
max,8.874137,16.239998,11.210007,3.930395e+08,0.239594,0.071043,8.874137,14.070968,97.575313,95.729289,...,100.000000,100.000000,368.904712,7.390591,25.016003,517.276520,1.636501,3.804334,4.703747,27.146007


In [32]:
train_ratio = 0.9
train_len = int(len(stock_slice)*train_ratio)
train_data = stock_slice[:train_len]
train_label = label_7[:train_len]
test_data = stock_slice[train_len:]
test_label = label_7[train_len:]

min_max_scaler = MinMaxScaler()
x_train = min_max_scaler.fit_transform(train_data)
x_test = min_max_scaler.transform(test_data)

In [38]:
clfRandomForest = RandomForestClassifier(n_estimators=1000,oob_score=True,n_jobs=-1, max_depth=7)
clfRandomForest = clfRandomForest.fit(train_data, train_label)

preds_train = clfRandomForest.predict(train_data)
preds_test = clfRandomForest.predict(test_data)
print(str(accuracy_score(preds_train, train_label)*100))
print(str(accuracy_score(preds_test, test_label)*100))
print(clfRandomForest.feature_importances_)

76.9607843137255
57.599999999999994
[0.01653413 0.0243304  0.01900766 0.02226461 0.03923055 0.02072079
 0.01761371 0.03604524 0.02552548 0.02596282 0.02663962 0.04072421
 0.05062301 0.04061326 0.03507016 0.04077075 0.05565326 0.04235866
 0.02404701 0.02292812 0.02379035 0.12879375 0.04329896 0.04200967
 0.0438437  0.03019548 0.02891817 0.03248649]


In [ ]:
train_data = stock_slice[:4000]
train_label = label_1[:4000]
test_data = stock_slice[4000:]
test_label = label_1[4000:]

min_max_scaler = MinMaxScaler()
x_train = min_max_scaler.fit_transform(train_data)
x_test = min_max_scaler.transform(test_data)

clfXGB = XGBClassifier(n_estimator=100,max_depth=5,gamma=0.1,colsample_bytree=0.5,subsample=0.5,max_delta_step=500)
clfXGB = clfXGB.fit(x_train, train_label)

preds_train = clfXGB.predict(x_train)
preds_test = clfXGB.predict(x_test)
print(str(accuracy_score(preds_train, train_label)*100))
print(str(accuracy_score(preds_test, test_label)*100))
print(clfXGB.feature_importances_)

In [59]:
train_data = stock_slice[['macdh', 'atr', 'vr']][:4000]
train_label = label_7[:4000]
test_data = stock_slice[['macdh', 'atr', 'vr']][4000:]
test_label = label_7[4000:]

min_max_scaler = MinMaxScaler()
x_train = min_max_scaler.fit_transform(train_data)
x_test = min_max_scaler.transform(test_data)

eval_set = [(x_test, test_label)]
clfXGB = XGBClassifier(n_estimator=500,max_depth=5,gamma=0.1,colsample_bytree=0.7)
clfXGB = clfXGB.fit(x_train, train_label)

preds_train = clfXGB.predict(x_train)
preds_test = clfXGB.predict(x_test)
print(str(accuracy_score(preds_train, train_label)*100))
print(str(accuracy_score(preds_test, test_label)*100))
print(clfXGB.feature_importances_)

76.175
57.16129032258065
[0.30069524 0.36674392 0.33256084]


In [1]:
train_data = stock_slice[:2000]
train_label = label_30[:2000]
test_data = stock_slice[2000:]
test_label = label_30[2000:]

min_max_scaler = MinMaxScaler()
x_train = min_max_scaler.fit_transform(train_data)
x_test = min_max_scaler.transform(test_data)

clfXGB = XGBClassifier(n_estimator=1000,max_depth=6,gamma=0.1,min_child_weight=1,colsample_bytree=0.5,subsample=0.5,max_delta_step=100)
clfXGB = clfXGB.fit(x_train, train_label)

preds_train = clfXGB.predict(x_train)
preds_test = clfXGB.predict(x_test)
print(str(accuracy_score(preds_train, train_label)*100))
print(str(accuracy_score(preds_test, test_label)*100))
print(pd.DataFrame(clfXGB.feature_importances_, columns=stock_slice.columns))

NameError: name 'stock_slice' is not defined

In [ ]:
pd.DataFrame(x_train).describe()

In [ ]:
# 加入大盤因素 NYSE, S&P 指數

# Random Forest frequent usage and test

In [211]:
# feature_importance
rfc_model.best_estimator_.feature_importances_
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(stock_with_absolute.columns, rfc_model.best_estimator_.feature_importances_):
    feats[feature] = importance #add the name/value pair 
    
with open('./results/RandomForest_with_ABS_pred_1_d.txt', 'w') as file:
     file.write(json.dumps(feats))
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances = importances.sort_values(by='Gini-importance', ascending=False)
importances.to_csv('./results/RandomForest_with_ABS_pred_1_d_feature_importance.csv')

In [212]:
importances.to_csv('./results/RandomForest_with_ABS_pred_1_d_feature_importance.csv')

In [163]:
train_ratio = 0.9
train_len = int(len(stock_with_absolute)*train_ratio)
train_data = stock_with_absolute[:train_len]
train_label = label_abs_1d[:train_len]
test_data = stock_with_absolute[train_len:]
test_label = label_abs_1d[train_len:]

min_max_scaler = MinMaxScaler()
x_train = min_max_scaler.fit_transform(train_data)
x_test = min_max_scaler.fit_transform(test_data)

In [169]:
clfRandomForest = RandomForestClassifier(n_estimators=1000, n_jobs=-1, max_depth=2)
clfRandomForest = clfRandomForest.fit(train_data, train_label)

preds_train = clfRandomForest.predict(train_data)
preds_test = clfRandomForest.predict(test_data)
print(str(accuracy_score(preds_train, train_label)*100))
print(str(accuracy_score(preds_test, test_label)*100))
print(clfRandomForest.feature_importances_)

%matplotlib inline
#do code to support model
#"data" is the X dataframe and model is the SKlearn object

feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(stock_with_absolute.columns, clfRandomForest.feature_importances_):
    feats[feature] = importance #add the name/value pair 

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances = importances.sort_values(by='Gini-importance', ascending=False)
top_20 = importances.sort_values(by='Gini-importance', ascending=False).head(20)
top_10 = importances.sort_values(by='Gini-importance', ascending=False).head(10)
top_5 = importances.sort_values(by='Gini-importance', ascending=False).head(5)
top_1 = importances.sort_values(by='Gini-importance', ascending=False).head(1)

58.013544018058695
55.87044534412956
[0.00383867 0.01141777 0.01052736 0.0065471  0.00792823 0.01294354
 0.00371414 0.01544812 0.00886071 0.00351098 0.00559752 0.00383123
 0.00846357 0.00874588 0.03291526 0.02504628 0.0429938  0.02254598
 0.03556545 0.01621474 0.00387701 0.01506438 0.02457665 0.00738584
 0.01663393 0.00689802 0.02138222 0.02667249 0.01026082 0.00916457
 0.00956482 0.01020405 0.01150361 0.00569984 0.00979025 0.00651245
 0.00550537 0.00565461 0.00588129 0.005182   0.00360406 0.01153507
 0.01147091 0.00945782 0.00569458 0.01558307 0.01548595 0.01300079
 0.00392209 0.00914889 0.00838922 0.01695401 0.01585222 0.01087385
 0.0158528  0.0122826  0.00516181 0.003707   0.00919831 0.00421643
 0.00316082 0.00561683 0.00602569 0.00679927 0.00372796 0.02772358
 0.00606061 0.00686804 0.00647367 0.01035193 0.00857691 0.00742821
 0.00777176 0.00710457 0.01056099 0.01162772 0.00635932 0.01243519
 0.01469418 0.00646873 0.00496003 0.00450358 0.00537678 0.00486666
 0.01398097 0.01267969 0.

In [170]:
top_features = stock_with_absolute[top_5.index]
train_ratio = 0.9
train_len = int(len(stock_with_absolute)*train_ratio)
train_data = top_features[:train_len]
train_label = label_abs_1d[:train_len]
test_data = top_features[train_len:]
test_label = label_abs_1d[train_len:]

min_max_scaler = MinMaxScaler()
x_train = min_max_scaler.fit_transform(train_data)
x_test = min_max_scaler.transform(test_data)

In [172]:
clfRandomForest = RandomForestClassifier(n_estimators=1000, n_jobs=-1, max_depth=5)
clfRandomForest = clfRandomForest.fit(train_data, train_label)

preds_train = clfRandomForest.predict(train_data)
preds_test = clfRandomForest.predict(test_data)
print(str(accuracy_score(preds_train, train_label)*100))
print(str(accuracy_score(preds_test, test_label)*100))
print(clfRandomForest.feature_importances_)

%matplotlib inline
#do code to support model
#"data" is the X dataframe and model is the SKlearn object

feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(top_features.columns, clfRandomForest.feature_importances_):
    feats[feature] = importance #add the name/value pair 

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances = importances.sort_values(by='Gini-importance', ascending=False)
importances

65.95936794582393
51.417004048582996
[0.20476452 0.20136312 0.20308733 0.20054698 0.19023805]


,Gini-importance
wr_6,0.204765
rsi_6,0.203087
cci,0.201363
gold_close_-6_r,0.200547
open_close_diff,0.190238
